# CPU vs GPU benchmark

CPU를 사용할 때와 GPU를 사용할 때 모델의 학습 속도를 비교한다.

## 아래 코드는 현재 GPU를 확인한다
빈칸이면 GPU가 없는 것.

In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

### ※ 참고

`Device(CPU/GPU)`는 텐서를 할당할때부터 사용된다. 따라서 학습데이터와 모델을 정의하는 과정부터 CPU/GPU별로 따로 실행해야한다.

그래서 **함수**로 만들어놓고 두번 호출하도록 했다.

## 데이터를 로드하고 전처리

In [2]:
def prepare_data():
    (X_train, y_train), (_, _) = tf.keras.datasets.cifar10.load_data()

    X_train = X_train.astype('float32') / 255.  # 0~1 사이 값으로 정규화
    
    return X_train, y_train

## 일반 MLP와 CNN 모델 만들기

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

def build_models():
    MLP = Sequential([
        Flatten(input_shape=(32, 32, 3)),
        Dense(256, activation='relu'),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ], name='MLP')

    CNN = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(32, 32, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(10, activation="softmax"),
    ], name='CNN')
    
    return MLP, CNN

## CPU와 GPU에서 각각 훈련해보기

In [4]:
%%time
with tf.device('/CPU:0'):
    X_train, y_train = prepare_data()
    MLP, CNN = build_models()
    # y = [0] ~ [9] 그러므로, sparse_categorical_crossentropy 사용.
    MLP.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    CNN.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    print("* Starting MLP fit...")
    MLP.fit(X_train, y_train, epochs=3)
    print("* Starting CNN fit...")
    CNN.fit(X_train, y_train, epochs=3)
    print()

* Starting MLP fit...
Epoch 1/3
1563/1563 [==============================] - 4s 2ms/step - loss: 1.8862 - accuracy: 0.3184
Epoch 2/3
1563/1563 [==============================] - 4s 2ms/step - loss: 1.7082 - accuracy: 0.3899
Epoch 3/3
1563/1563 [==============================] - 4s 2ms/step - loss: 1.6301 - accuracy: 0.4180
* Starting CNN fit...
Epoch 1/3
1563/1563 [==============================] - 14s 9ms/step - loss: 1.4164 - accuracy: 0.4960
Epoch 2/3
1563/1563 [==============================] - 13s 9ms/step - loss: 1.0910 - accuracy: 0.6215
Epoch 3/3
1563/1563 [==============================] - 13s 9ms/step - loss: 0.9781 - accuracy: 0.6640

Wall time: 53.2 s


In [5]:
%%time
if tf.config.list_physical_devices("GPU"):
    with tf.device('/GPU:0'):
        X_train, y_train = prepare_data()
        MLP, CNN = build_models()
        # y = [0] ~ [9] 그러므로, sparse_categorical_crossentropy 사용.
        MLP.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        CNN.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        print("* Starting MLP fit...")
        MLP.fit(X_train, y_train, epochs=3)
        print("* Starting CNN fit...")
        CNN.fit(X_train, y_train, epochs=3)
        print()
else:
    print("No GPU. Skip.")

No GPU. Skip.
Wall time: 1 ms
